<a href="https://colab.research.google.com/github/pkw408/LLM/blob/main/20231018_translation_kw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
pip install googletrans==3.1.0a0

In [33]:
from googletrans import Translator

translator = Translator()
result = translator.translate("굴착기란 땅을 파거나 깎는 현장에서 사용되는 건설기계. 건설현장을 상징하는 장비로 굴착과 토사 운반은 물론 건물 해체와 지면 정리에도 투입된다.", dest="en")
print(result.text)


An excavator is a construction machine used at a site to dig or cut the ground. This equipment symbolizes construction sites and is used not only for excavation and transport of soil, but also for dismantling buildings and clearing the ground.


In [34]:
pip install python-pptx

In [51]:
from pptx import Presentation

# pptx의 text를 속성 그대로 살려두고 글자만 변경
def replace_paragraph_text_retaining_initial_formatting(paragraph, new_text):
    if len(paragraph.runs) == 0:
        return
    p = paragraph._p
    for idx, run in enumerate(paragraph.runs):
        if idx == 0:
            continue
        p.remove(run._r)
    paragraph.runs[0].text = new_text

def translate(shape, src_lang, tgt_lang):
    for paragraph in shape.text_frame.paragraphs:
        # 문자열 예외 처리 공백..
        if paragraph.text.isspace():
            continue
        if paragraph.text == '':
            continue
        # 문자열이 번역할 언어인지 확인
        translator = Translator() # translator 선언
        # print(translator.detect(paragraph.text))
        # print(src_lang)
        if translator.detect(paragraph.text).lang == src_lang:
            # tgt_lang 언어로 번역
            new_text = translator.translate(paragraph.text, src = src_lang, dest = tgt_lang)

            print(new_text)
            if new_text == None:
                return False
            # 문자열 표시
            print('src text :' + paragraph.text + ", target text : " + new_text.text)
            # pptx 해당 문자열 변경
            replace_paragraph_text_retaining_initial_formatting(paragraph, new_text.text)
    # 문제없이 번역되었다면 True 리턴
    return True

# 테이블 안에 있는 문자열 번역
def table_func(shape, src_lang, tgt_lang):
    table = shape.table
    # 셀단위로 체크
    for cell in table.iter_cells():
        if translate(cell, src_lang, tgt_lang) is False:
            return False
    return True

# 텍스트 상자 문자열 번역
def text_func(shape, src_lang, tgt_lang):
    return translate(shape, src_lang, tgt_lang)

def run(src_file, tgt_file, src_lang, tgt_lang):
    # pptx 불러오기
    prs = Presentation(src_file)
    # 슬라이드 단위로 루프
    for slide in prs.slides:
        # 슬라이드에서 각 객체(모양?) 단위로 루프
        for shape in slide.shapes:
            # 객체가 테이블이면
            if shape.has_table:
                # 테이블 번역 함수 호출
                if table_func(shape, src_lang, tgt_lang) is False:
                    prs.save(tgt_file)
                    return False
            # 텍스트 상자면
            if shape.has_text_frame:
                # 텍스트 번역 함수 호출
                if text_func(shape, src_lang, tgt_lang) is False:
                    prs.save(tgt_file)
                    return False
    # 끝까지 문제없이 번역되었다면 저장하고 리턴
    prs.save(tgt_file)
    return True

In [52]:
if __name__ == "__main__":
    src_file = './drive/MyDrive/연구분야(대학원).pptx' # 번역할 pptx파일
    tgt_file = './drive/MyDrive/translated.pptx' # 번역되어 저장할 pptx 파일
    src_lang = 'ko' # 번역할 언어
    tgt_lang = 'en' # 번역을 요청할 언어

    # src_lang = 'en' # 번역할 언어
    # tgt_lang = 'ko' # 번역을 요청할 언어

    # 번역 시작
    run(src_file, tgt_file, src_lang, tgt_lang)

Translated(src=ko, dest=en, text=Research content, pronunciation=None, extra_data="{'translat...")
src text :연구 내용, target text : Research content
Translated(src=ko, dest=en, text=01. Medical, pronunciation=None, extra_data="{'translat...")
src text :01. 의료, target text : 01. Medical
Translated(src=ko, dest=en, text=02. Security, pronunciation=None, extra_data="{'translat...")
src text :02. 보안, target text : 02. Security
Translated(src=ko, dest=en, text=03. Video abnormality detection, pronunciation=None, extra_data="{'translat...")
src text :03. 비디오 이상탐지, target text : 03. Video abnormality detection
Translated(src=ko, dest=en, text=04. Manufacturing, pronunciation=None, extra_data="{'translat...")
src text :04. 제조, target text : 04. Manufacturing
Translated(src=ko, dest=en, text=05. Graph, pronunciation=None, extra_data="{'translat...")
src text :05. 그래프 , target text : 05. Graph
Translated(src=ko, dest=en, text=Development of artificial intelligence model for gallbladder cancer diag

## 엑셀 문서 번역 (LLM 모델 활용)

In [37]:
!pip install transformers

In [38]:
# LLM 모델 불러오기
from transformers import pipeline
pipe = pipeline("translation", model="KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-bidirection")
### 이 모델이 어디서 다운 받았는지 - https://huggingface.co/KETI-AIR-Downstream (en2ko 도 존재)

In [39]:
#번역 시험
txt = '이번 세션은 LLM모델을 사용하여 엑셀 자동 번역기를 만들어 보겠습니다.'
pipe("translate_ko2en: " + txt, max_length=40)

[{'translation_text': 'In this session, we will use the LLM model to create an Excel automatic translator.'}]

In [40]:
type(pipe("translate_ko2en: " + txt, max_length=40))

list

In [41]:
#list 내부에 dictionary 형태로 출력됩니다. 번역 결과만 가져오기 위해 아래 함수를 정의합니다.

def translate(text, maxl=40):
  trans = pipe("translate_ko2en: " + text, max_length=maxl)[0]["translation_text"]
  return trans

In [42]:
# !pip install openpyxl
# !sudo apt-get install python3-tk  # 왜 Sudo???

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [44]:
from openpyxl import load_workbook, Workbook
import datetime

#번역할 파일업로드 후, 선택합니다
files = './drive/MyDrive/220914_conferenceDeadline_v2.xlsx'
load_wb = load_workbook(files, data_only=True)
ws = load_wb.active
result_wb = Workbook()
result_ws = result_wb.active

In [45]:
#엑셀 파일의 최대 행과 열을 조회합니다.
row = ws.max_row
col = ws.max_column
print('행 :',row, ', 열 :',col)

행 : 30 , 열 : 6


In [46]:
# 각 셀의 내용을 번역기에 넣어주어야 합니다.
for j in range(col):
  for i in range(row):
    string = ws.cell(i+1, j+1).value   # 해당 셀의 내용을 읽어옵니다.
    if type(string) == str:   # 데이터 타입이 문자인 것만 번역합니다.
      result = translate(string)
      result_ws.cell(i+1, j+1).value = result
      print(string,' --> ', result)
    else:   # 데이터 타입이 문자가 아니라면 번역하지 않습니다.
      result_ws.cell(i+1, j+1).value = string
      print(string, ' not changed')
    today = datetime.datetime.now()
    today_md = today.strftime("%y%m%d") + '_' + today.strftime("%H%M") + "_translated"

save_dir = '/content/' + today_md + '.xlsx'
result_wb.save(save_dir)

약칭  -->  Abbreviated
CVPR  -->  CVPR
ICCV  -->  ICCV
SIGMOD  -->  SIGMOD
ICDE  -->  ICDE
ICDM  -->  ICDM
ICASSP  -->  ICASSP
KSC  -->  KSC
ICLR  -->  ICLR
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
None  not changed
기타 일정  -->  Other schedules
None  not changed
None  not changed
학술대회명  -->  The name of the conference
Conference on Computer Vision and Pattern Recognition  -->  Computer vision and pattern recognition conference
International Conference on Computer Vision  -->  International Conference on Computer Vision
International Conference on Management of Data  -->  International Conference on Data Management
IEEE International Conference on Data Engineering  -->  IEEE International Conference on Data Engineering
IEEE Inter

In [ ]:
## 파일다운로드 하는법 : https://lapina.tistory.com/140
# https://github.com/wkentaro/gdown